### Pet Classification Model Using CNN

#### Project Objective:


Build a CNN model that classifies the given pet images correctly into dog and cat images. 
The project scope document specifies the requirements for the project “Pet Classification Model Using CNN.” Apart from specifying the functional and non-functional requirements for the project, it also serves as an input for project scoping. 


#### Project Description and Scope: 


You are provided with a collection of images of pets, that is, cats and dogs. These images are of different sizes with varied lighting conditions and they should be used as inputs for your model.

You are expected to write the code for CNN image classification model using TensorFlow that trains on the data and calculates the accuracy score on the test data. 


#### Project Guidelines:


Begin by creating the ipynb file in the same parent folder where the downloaded data set is kept. The CNN model should have the following layers:<br>
● Input layer <br>
● Convolutional layer 1 with 32 filters of kernel size[5,5] <br>
● Pooling layer 1 with pool size[2,2] and stride 2 <br>
● Convolutional layer 2 with 64 filters of kernel size[5,5] <br>
● Pooling layer 2 with pool size[2,2] and stride 2 <br>
● Dense layer whose output size is fixed in the hyper parameter: fc_size=32 <br>
● Dropout layer with dropout probability 0.4 <br>

Predict the class by doing a softmax on the output of the dropout layers. This should be followed by training and evaluation: <br>
● For the training step, define the loss function and minimize it <br>
● For the evaluation step, calculate the accuracy <br>

Run the program for 100, 200, and 300 iterations, respectively. Follow this by a report on the final accuracy and loss on the evaluation data. 



In [33]:
#For seeding (set random seed for numpy and tensorflow backend):
from numpy.random import seed
seed(42)
import random
import tensorflow as tf
tf.random.set_seed(69)

# Keras & Tensorflow for building CNN model
# import tensorflow_datasets as tfds
from tensorflow.keras import layers
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from keras.preprocessing.image import ImageDataGenerator

# General:
import numpy as np
import cv2                 # for working with images
import os                  # for dealing with directories
import warnings
warnings.filterwarnings('ignore')

In [34]:
# Notes on Seeding:
# In neural networks weights are initialized randomly. This happens using random number generator which will generate 
# numbers randomly every time you initialize the neural network. The problem with this is that your results will be 
# different every time you initialize training process of network. Thus, in order to get reproducible results for research 
# or production, you need to fix random seed so that you can experiment with other parts of network such as number of neurons, 
# hidden layers etc. without worrying about random number generator which can vary weights changing network dynamics.

#### Data Preparation

In [35]:
img_shape=200
train_dir = 'D:\\Simplilearn\\04 Deep Learning with Keras and Tensorflow\\Assignment\\data\\train'
test_dir = 'D:\\Simplilearn\\04 Deep Learning with Keras and Tensorflow\\Assignment\\data\\test'

In [36]:
# Using ImageDataGenerator to normalize and transform the images

train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    # We divide each pixel value(0, 255) with 255 to make them in range [0, 1]
    rescale=1./255, 
    # We randomly shear & zoom our image while training to make our training robust
    shear_range = 0.2, 
    zoom_range = 0.2,
    # We also flip our images by 180 degree horizontally to make our training robust
    horizontal_flip = True 
)


test_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
)

In [37]:
# Read images folder and apply transormation while doing so:

training_set_data = train_data_generator.flow_from_directory(
    train_dir, 
    target_size = (img_shape, img_shape), 
    batch_size = 64,
    class_mode = 'binary'
)

test_set_data = test_data_generator.flow_from_directory(
    test_dir,
    target_size = (img_shape, img_shape),
    batch_size = 64, 
    class_mode = 'binary'
)

Found 40 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


#### Build CNN Model

The CNN model should have the following layers:<br><br>
● Input layer <br>
● Convolutional layer 1 with 32 filters of kernel size[5,5] <br>
● Pooling layer 1 with pool size[2,2] and stride 2 <br>
● Convolutional layer 2 with 64 filters of kernel size[5,5] <br>
● Pooling layer 2 with pool size[2,2] and stride 2 <br>
● Dense layer whose output size is fixed in the hyper parameter: fc_size=32 <br>
● Dropout layer with dropout probability 0.4 <br><br>
Predict the class by doing a softmax on the output of the dropout layers

In [38]:
fc_size=32
dropout=0.4

In [39]:
# Input layer
input_layer = layers.Input(shape=(img_shape, img_shape, 3))
x = input_layer

# Convolutional layer 1 with 32 filters of kernel size[5,5]
x = layers.Conv2D(32, kernel_size=(5,5), activation = 'relu')(x)
# Pooling layer 1 with pool size[2,2] and stride 2
x = layers.MaxPooling2D(pool_size=(2,2), strides=2)(x)

# Convolutional layer 2 with 64 filters of kernel size[5,5]
x = layers.Conv2D(64, kernel_size=(5,5), activation = 'relu')(x)
# Pooling layer 2 with pool size[2,2] and stride 2
x = layers.MaxPooling2D(pool_size=(2,2),strides=2)(x)

# Dense layer whose output size is fixed in the hyper parameter: fc_size=32
x = layers.Flatten()(x)
x = layers.Dense(fc_size, activation='relu')(x)
# Dropout layer with dropout probability 0.4
x = layers.Dropout(dropout)(x)

# Predict the class by doing a softmax on the output of the dropout layers
x = layers.Dense(1, activation='softmax')(x)

In [40]:
# model_1 - Epoch 100
model_1 = keras.Model(inputs = input_layer, outputs = x)
model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_1.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 196, 196, 32)      2432      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 98, 98, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 94, 94, 64)        51264     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 47, 47, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 141376)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                4524

In [41]:
# model_2 - Epoch 200
model_2 = keras.Model(inputs = input_layer, outputs = x)
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_2.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 196, 196, 32)      2432      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 98, 98, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 94, 94, 64)        51264     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 47, 47, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 141376)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                4524

In [42]:
# model_3 - Epoch 300
model_3 = keras.Model(inputs = input_layer, outputs = x)
model_3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_3.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 196, 196, 32)      2432      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 98, 98, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 94, 94, 64)        51264     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 47, 47, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 141376)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                4524

#### Fitting data with the CNN Models – 100, 200 and 300 epochs

##### 100 Epochs

In [43]:
model_1.fit(
    training_set_data, 
    epochs = 100, 
    validation_data = test_set_data, 
)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 0.6971 - accuracy: 0.5000 - val_loss: 0.8736 - val_accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 2s 2s/step - loss: 0.9748 - accuracy: 0.5000 - val_loss: 1.6126 - val_accuracy: 0.5000
Epoch 3/100
1/1 [==============================] - 2s 2s/step - loss: 1.5398 - accuracy: 0.5000 - val_loss: 0.6966 - val_accuracy: 0.5000
Epoch 4/100
1/1 [==============================] - 2s 2s/step - loss: 0.7853 - accuracy: 0.5000 - val_loss: 0.7053 - val_accuracy: 0.5000
Epoch 5/100
1/1 [==============================] - 2s 2s/step - loss: 0.6801 - accuracy: 0.5000 - val_loss: 0.6794 - val_accuracy: 0.5000
Epoch 6/100
1/1 [==============================] - 2s 2s/step - loss: 0.6760 - accuracy: 0.5000 - val_loss: 0.6653 - val_accuracy: 0.5000
Epoch 7/100
1/1 [==============================] - 2s 2s/step - loss: 0.6957 - accuracy: 0.5000 - val_loss: 0.6556 - val_accuracy: 0.5000
Epoch 8/100
1/1 [=================

In [44]:
score_100 = model_1.evaluate(test_set_data)
print('\nTest loss for 100 epochs: %0.2f%%' % (score_100[0] * 100))
print('\nTest accuracy for 100 epochs: %0.2f%%' % (score_100[1] * 100))

1/1 [==============================] - 0s 214ms/step - loss: 2.5711 - accuracy: 0.5000

Test loss for 100 epochs: 257.11%

Test accuracy for 100 epochs: 50.00%


##### 200 Epochs

In [45]:
model_2.fit(
    training_set_data, 
    epochs = 200, 
    validation_data = test_set_data, 
)

Epoch 1/200
1/1 [==============================] - 2s 2s/step - loss: 0.0184 - accuracy: 0.5000 - val_loss: 8.2369 - val_accuracy: 0.5000
Epoch 2/200
1/1 [==============================] - 2s 2s/step - loss: 2.1488 - accuracy: 0.5000 - val_loss: 3.3867 - val_accuracy: 0.5000
Epoch 3/200
1/1 [==============================] - 2s 2s/step - loss: 0.0144 - accuracy: 0.5000 - val_loss: 2.6909 - val_accuracy: 0.5000
Epoch 4/200
1/1 [==============================] - 2s 2s/step - loss: 0.0751 - accuracy: 0.5000 - val_loss: 2.2708 - val_accuracy: 0.5000
Epoch 5/200
1/1 [==============================] - 2s 2s/step - loss: 0.5305 - accuracy: 0.5000 - val_loss: 1.7216 - val_accuracy: 0.5000
Epoch 6/200
1/1 [==============================] - 2s 2s/step - loss: 0.3496 - accuracy: 0.5000 - val_loss: 1.3574 - val_accuracy: 0.5000
Epoch 7/200
1/1 [==============================] - 2s 2s/step - loss: 0.3548 - accuracy: 0.5000 - val_loss: 1.1255 - val_accuracy: 0.5000
Epoch 8/200
1/1 [=================

In [46]:
score_200 = model_2.evaluate(test_set_data)
print('\nTest loss for 200 epochs: %0.2f%%' % (score_200[0] * 100))
print('\nTest accuracy for 200 epochs: %0.2f%%' % (score_200[1] * 100))

1/1 [==============================] - 0s 207ms/step - loss: 5.0606 - accuracy: 0.5000

Test loss for 200 epochs: 506.06%

Test accuracy for 200 epochs: 50.00%


##### 300 Epochs

In [47]:
model_3.fit(
    training_set_data, 
    epochs = 300, 
    validation_data = test_set_data, 
)

Epoch 1/300
1/1 [==============================] - 2s 2s/step - loss: 0.0012 - accuracy: 0.5000 - val_loss: 5.4423 - val_accuracy: 0.5000
Epoch 2/300
1/1 [==============================] - 2s 2s/step - loss: 0.2426 - accuracy: 0.5000 - val_loss: 5.2620 - val_accuracy: 0.5000
Epoch 3/300
1/1 [==============================] - 2s 2s/step - loss: 0.0042 - accuracy: 0.5000 - val_loss: 7.5960 - val_accuracy: 0.5000
Epoch 4/300
1/1 [==============================] - 2s 2s/step - loss: 0.6837 - accuracy: 0.5000 - val_loss: 5.1293 - val_accuracy: 0.5000
Epoch 5/300
1/1 [==============================] - 2s 2s/step - loss: 0.0210 - accuracy: 0.5000 - val_loss: 4.5352 - val_accuracy: 0.5000
Epoch 6/300
1/1 [==============================] - 2s 2s/step - loss: 0.0076 - accuracy: 0.5000 - val_loss: 4.2556 - val_accuracy: 0.5000
Epoch 7/300
1/1 [==============================] - 2s 2s/step - loss: 0.0271 - accuracy: 0.5000 - val_loss: 4.1743 - val_accuracy: 0.5000
Epoch 8/300
1/1 [=================

In [48]:
score_300 = model_3.evaluate(test_set_data)
print('\nTest loss for 300 epochs: %0.2f%%' % (score_300[0] * 100))
print('\nTest accuracy for 300 epochs: %0.2f%%' % (score_300[1] * 100))

1/1 [==============================] - 0s 211ms/step - loss: 8.9512 - accuracy: 0.5000

Test loss for 300 epochs: 895.12%

Test accuracy for 300 epochs: 50.00%


In [52]:
print('\nAccuracy scores remain at 50% for each model despite the increase in Epochs. \n'+
'In fact, raising the Epochs appears to increase model loss. \n'+
'This may be occuring due to such a small dataset size')


Accuracy scores remain at 50% for each model despite the increase in Epochs. 
In fact, raising the Epochs appears to increase model loss. 
This may be occuring due to such a small dataset size


#### Miscellaneous (can ignore):

In [49]:
# training_set_data = tf.keras.utils.image_dataset_from_directory(
#   train_dir,
# #   validation_split=0.2,
# #   subset="training",
# #   seed=123,
#   image_size=(img_shape, img_shape),
#   batch_size=32,
# )

# test_set_data = tf.keras.utils.image_dataset_from_directory(
#   test_dir,
# #   validation_split=0.2,
# #   subset="validation",
# #   seed=123,
#   image_size=(img_shape, img_shape),
#   batch_size=32
# )

#===============================================================

# train_gen = train_data_generator.flow_from_directory(
#     train_dir, 
#     target_size=(img_shape, img_shape),
#     batch_size=32)

# training_set_data = tf.data.Dataset.from_generator(
#     lambda: train_gen,
#     output_types = (tf.float32, tf.float32),
#     output_shapes = ([None, img_shape, img_shape, 3], [None, 1]),
# )

# # =========================================================================

# test_gen = test_data_generator.flow_from_directory(
#     test_dir, 
#     target_size=(img_shape, img_shape),
#     batch_size=32)

# test_set_data = tf.data.Dataset.from_generator(
#     lambda: test_gen,
#     output_types = (tf.float32, tf.float32),
#     output_shapes = ([None, img_shape, img_shape, 3], [None, 1]),
# )

# # ========================================================